In [1]:
# CELL 1: Import all necessary libraries
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import joblib
import json
from datetime import datetime
import warnings
import os

# Set professional plot style
sns.set_theme(style="whitegrid")
warnings.filterwarnings("ignore", category=UserWarning)

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

print("✅ Libraries imported successfully!")
print("�� Ready to train GMM models on ALL 34 combinations!")

✅ Libraries imported successfully!
�� Ready to train GMM models on ALL 34 combinations!


In [2]:
# CELL 2: Database connection and load ALL data
config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'root',
    'database': 'spherych_devapp'
}

# Load the comprehensive dataset with ALL game modes
complete_query = """
SELECT 
    w.id as workout_id,
    w.score,
    w.completedWorkout,
    w.measuredDuration as actual_duration,
    w.userId,
    
    -- Game Mode Information
    wp.name as game_mode,
    wp.exercisePool,
    
    -- Race Configuration
    rc.difficulty,
    rc.duration as planned_duration,
    rc.startSpeed,
    rc.hrTarget,
    
    -- User Demographics
    u.username,
    hd.dob,
    hd.weight,
    hd.height
    
FROM Workouts w
JOIN WorkoutPresets wp ON w.workoutPresetId = wp.id
JOIN RaceConfigs rc ON w.id = rc.workoutId
JOIN Users u ON w.userId = u.id
LEFT JOIN HealthData hd ON u.id = hd.userId

WHERE w.completedWorkout = 1 AND w.score > 0
"""

try:
    print("🔄 Loading comprehensive dataset...")
    conn = mysql.connector.connect(**config)
    df_complete = pd.read_sql(complete_query, conn)
    print(f"✅ Successfully loaded {len(df_complete)} records.")
    
    # Show game modes available
    print("\n🎮 Game modes found:")
    print(df_complete['game_mode'].value_counts())
    
except Exception as e:
    print(f"❌ Error: {e}")
    df_complete = None
finally:
    if 'conn' in locals() and conn.is_connected():
        conn.close()

🔄 Loading comprehensive dataset...
✅ Successfully loaded 11448 records.

🎮 Game modes found:
game_mode
DualFlow               5780
UpperBody              2551
LeagueQualification    2202
RehaFlow                827
LegDay                   78
HomeFlow                 10
Name: count, dtype: int64


In [3]:
# CELL 3: Feature engineering for ALL data
if df_complete is not None:
    print("�� Performing feature engineering...")
    
    # Calculate age from dob
    dob_datetime = pd.to_datetime(df_complete['dob'], errors='coerce')
    df_complete['age'] = (pd.to_datetime('today') - dob_datetime).dt.days / 365.25
    
    # Convert duration from seconds to minutes
    df_complete['duration_minutes'] = df_complete['planned_duration'] / 60
    
    # Create age groups
    bins = [0, 18, 25, 35, 45, 55, 100]
    labels = ['<18', '18-25', '26-35', '36-45', '46-55', '55+']
    df_complete['age_group'] = pd.cut(df_complete['age'], bins=bins, labels=labels, right=False)
    
    print(f"✅ Feature engineering completed!")
    print(f"�� Dataset shape: {df_complete.shape}")
    print(f"\n�� Summary statistics:")
    display(df_complete[['score', 'difficulty', 'duration_minutes', 'startSpeed', 'age']].describe())
    
    # Check for missing values
    print(f"\n�� Missing values:")
    print(df_complete[['score', 'age', 'startSpeed', 'duration_minutes']].isnull().sum())
else:
    print("❌ No data loaded. Please check CELL 2.")

�� Performing feature engineering...
✅ Feature engineering completed!
�� Dataset shape: (11448, 18)

�� Summary statistics:


,score,difficulty,duration_minutes,startSpeed,age
count,1.144800e+04,11448.000000,11448.000000,11448.000000,9451.000000
mean,1.936351e+05,0.636705,9.572196,6.128931,44.960227
std,1.973387e+05,1.385635,6.047691,2.381418,14.566403
min,1.000000e+00,-1.000000,0.000000,0.000000,0.772074
25%,5.204125e+04,-1.000000,5.000000,5.000000,34.699521
50%,1.296430e+05,1.000000,10.000000,5.000000,44.134155
75%,2.833750e+05,2.000000,10.000000,8.000000,55.808350
max,1.395700e+06,2.000000,30.000000,10.000000,77.352498



�� Missing values:
score                  0
age                 1997
startSpeed             0
duration_minutes       0
dtype: int64


In [4]:
# CELL 4: Find ALL viable combinations for modeling
if df_complete is not None:
    print("�� Finding ALL viable combinations for modeling...")
    
    # Create combination analysis
    combination_analysis = df_complete.groupby(['game_mode', 'difficulty', 'duration_minutes']).agg({
        'workout_id': 'count',
        'score': ['mean', 'std', 'min', 'max'],
        'startSpeed': 'mean'
    }).round(2)
    
    combination_analysis.columns = ['workout_count', 'score_mean', 'score_std', 'score_min', 'score_max', 'avg_start_speed']
    combination_analysis = combination_analysis.reset_index()
    
    # Filter for combinations with at least 100 workouts
    viable_combinations = combination_analysis[combination_analysis['workout_count'] >= 100].sort_values('workout_count', ascending=False)
    
    print(f"✅ Found {len(viable_combinations)} viable combinations (≥100 workouts):")
    display(viable_combinations)
    
    # Store for later use
    all_combinations = viable_combinations.copy()
    
    print(f"\n📊 Summary:")
    print(f"- Total combinations: {len(viable_combinations)}")
    print(f"- Total workouts: {viable_combinations['workout_count'].sum():,}")
    print(f"- Average workouts per combination: {viable_combinations['workout_count'].mean():.1f}")
    
else:
    print("❌ No data available. Please check previous cells.")

�� Finding ALL viable combinations for modeling...
✅ Found 34 viable combinations (≥100 workouts):


,game_mode,difficulty,duration_minutes,workout_count,score_mean,score_std,score_min,score_max,avg_start_speed
2,DualFlow,-1,10.0,945,144742.74,74946.03,150,318100,5.44
26,DualFlow,2,10.0,919,170480.24,102694.25,50,368400,6.23
43,LeagueQualification,2,10.0,774,317667.12,118113.35,900,456150,6.76
25,DualFlow,2,5.0,581,72085.29,50678.47,550,178110,6.51
1,DualFlow,-1,5.0,489,60032.74,37587.16,400,139450,5.93
3,DualFlow,-1,15.0,472,247628.36,114103.47,60,480700,5.44
24,DualFlow,2,3.0,431,48080.05,28060.69,172,97580,8.18
90,UpperBody,2,5.0,415,83994.22,50888.10,250,185400,6.16
35,LeagueQualification,-1,5.0,362,99309.67,62593.30,500,200550,4.92
27,DualFlow,2,15.0,352,367455.32,151185.00,308,577400,7.47



📊 Summary:
- Total combinations: 34
- Total workouts: 10,168
- Average workouts per combination: 299.1


In [5]:
# CELL 5: Define function for granular percentile targets (every 5%)
def calculate_granular_percentiles(scores):
    """
    Calculate granular percentile targets every 5%
    
    Args:
        scores: Array of player scores
    
    Returns:
        Dictionary with percentile targets
    """
    # Define granular percentiles (every 5%)
    percentiles = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]
    
    # Calculate percentile values
    percentile_values = np.percentile(scores, percentiles)
    
    # Create dictionary with percentile targets
    targets = {}
    for p, value in zip(percentiles, percentile_values):
        targets[f'{p}th'] = int(value)
    
    return targets

print("✅ Granular percentile function defined!")
print("📊 Will calculate targets for: 5%, 10%, 15%, 20%, 25%, 30%, 35%, 40%, 45%, 50%, 55%, 60%, 65%, 70%, 75%, 80%, 85%, 90%, 95%")

✅ Granular percentile function defined!
📊 Will calculate targets for: 5%, 10%, 15%, 20%, 25%, 30%, 35%, 40%, 45%, 50%, 55%, 60%, 65%, 70%, 75%, 80%, 85%, 90%, 95%


In [8]:
# CELL 6: FIXED - Define function to train GMM for a specific combination
def train_gmm_for_combination(df_complete, game_mode, difficulty, duration_minutes, n_components=4):
    """
    Train GMM model for a specific game mode combination
    
    Args:
        df_complete: Complete dataset
        game_mode: Game mode name
        difficulty: Difficulty level
        duration_minutes: Duration in minutes
        n_components: Number of GMM components
    
    Returns:
        Dictionary with model data and results
    """
    try:
        # Filter for specific combination
        target_data = df_complete[
            (df_complete['game_mode'] == game_mode) &
            (df_complete['difficulty'] == difficulty) &
            (df_complete['duration_minutes'] == duration_minutes)
        ].copy()
        
        if len(target_data) < 100:
            return None
        
        # Prepare features for GMM
        features = ['score', 'age', 'startSpeed', 'duration_minutes']
        
        # IMPORTANT FIX: Handle missing values properly
        # First, get the data without missing values for training
        X = target_data[features].dropna()
        
        if len(X) < 50:  # Need minimum data for GMM
            return None
        
        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Train GMM
        gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=10)
        gmm.fit(X_scaled)
        
        # Get cluster assignments for the clean data
        cluster_labels = gmm.predict(X_scaled)
        cluster_probs = gmm.predict_proba(X_scaled)
        
        # FIXED: Create a new dataframe with only the rows that have complete data
        target_data_clean = target_data[features].dropna()
        target_data_clean = target_data_clean.reset_index(drop=True)
        
        # Add cluster information to the clean data
        target_data_clean['cluster'] = cluster_labels
        target_data_clean['cluster_probability'] = np.max(cluster_probs, axis=1)
        
        # Calculate cluster statistics using the clean data
        cluster_stats = target_data_clean.groupby('cluster').agg({
            'score': ['count', 'mean', 'std', 'min', 'max'],
            'age': 'mean',
            'startSpeed': 'mean',
            'cluster_probability': 'mean'
        }).round(2)
        
        cluster_stats.columns = ['count', 'score_mean', 'score_std', 'score_min', 'score_max', 
                               'avg_age', 'avg_start_speed', 'avg_confidence']
        
        # Calculate granular percentile targets for each cluster
        percentile_targets = {}
        for cluster in range(n_components):
            cluster_scores = target_data_clean[target_data_clean['cluster'] == cluster]['score']
            if len(cluster_scores) > 0:  # Only calculate if cluster has data
                percentile_targets[cluster] = calculate_granular_percentiles(cluster_scores)
            else:
                # If cluster is empty, use overall percentiles
                percentile_targets[cluster] = calculate_granular_percentiles(target_data_clean['score'])
        
        # Prepare model data
        model_data = {
            'gmm_model': gmm,
            'scaler': scaler,
            'features': features,
            'cluster_stats': cluster_stats.to_dict(),
            'percentile_targets': percentile_targets,
            'training_date': datetime.now().isoformat(),
            'dataset_info': {
                'total_records': len(target_data_clean),  # Use clean data count
                'original_records': len(target_data),      # Original count
                'dropped_records': len(target_data) - len(target_data_clean),  # How many dropped
                'game_mode': game_mode,
                'difficulty': difficulty,
                'duration_minutes': duration_minutes
            }
        }
        
        return model_data
        
    except Exception as e:
        print(f"❌ Error training GMM for {game_mode} - {difficulty} - {duration_minutes}min: {e}")
        return None

print("✅ FIXED GMM training function defined!")
print("🎯 Now handles missing values properly!")

✅ FIXED GMM training function defined!
🎯 Now handles missing values properly!


In [9]:
# CELL 7: FIXED - Train GMM models for ALL viable combinations
if 'all_combinations' in locals() and df_complete is not None:
    print("🚀 Starting GMM training for ALL combinations (FIXED VERSION)...")
    
    trained_models = {}
    successful_combinations = []
    failed_combinations = []
    
    for idx, row in all_combinations.iterrows():
        game_mode = row['game_mode']
        difficulty = row['difficulty']
        duration = row['duration_minutes']
        workout_count = row['workout_count']
        
        print(f"\n🔄 Training: {game_mode} - Difficulty {difficulty} - {duration}min ({workout_count} workouts)")
        
        # Train GMM for this combination
        model_data = train_gmm_for_combination(df_complete, game_mode, difficulty, duration)
        
        if model_data is not None:
            # Create model key
            model_key = f"{game_mode}_{difficulty}_{duration}min"
            trained_models[model_key] = model_data
            successful_combinations.append({
                'combination': model_key,
                'workout_count': model_data['dataset_info']['total_records'],
                'original_count': model_data['dataset_info']['original_records'],
                'dropped_count': model_data['dataset_info']['dropped_records'],
                'n_clusters': 4
            })
            
            print(f"✅ Successfully trained GMM for {model_key}")
            print(f"   📊 Clean records: {model_data['dataset_info']['total_records']}")
            print(f"   �� Dropped records: {model_data['dataset_info']['dropped_records']}")
        else:
            failed_combinations.append(f"{game_mode}_{difficulty}_{duration}min")
            print(f"❌ Failed to train GMM for {game_mode}_{difficulty}_{duration}min")
    
    print(f"\n🎯 TRAINING SUMMARY:")
    print(f"✅ Successful models: {len(successful_combinations)}")
    print(f"❌ Failed models: {len(failed_combinations)}")
    print(f"�� Total combinations attempted: {len(all_combinations)}")
    
    # Show successful combinations
    if successful_combinations:
        print(f"\n✅ Successfully trained models:")
        for combo in successful_combinations:
            print(f"  - {combo['combination']} ({combo['workout_count']} clean records, {combo['dropped_count']} dropped)")
    
    # Show failed combinations
    if failed_combinations:
        print(f"\n❌ Failed combinations:")
        for combo in failed_combinations:
            print(f"  - {combo}")
else:
    print("❌ No combinations or data available. Please run previous cells.")

🚀 Starting GMM training for ALL combinations (FIXED VERSION)...

🔄 Training: DualFlow - Difficulty -1 - 10.0min (945 workouts)
✅ Successfully trained GMM for DualFlow_-1_10.0min
   📊 Clean records: 855
   �� Dropped records: 90

🔄 Training: DualFlow - Difficulty 2 - 10.0min (919 workouts)
✅ Successfully trained GMM for DualFlow_2_10.0min
   📊 Clean records: 820
   �� Dropped records: 99

🔄 Training: LeagueQualification - Difficulty 2 - 10.0min (774 workouts)
✅ Successfully trained GMM for LeagueQualification_2_10.0min
   📊 Clean records: 567
   �� Dropped records: 207

🔄 Training: DualFlow - Difficulty 2 - 5.0min (581 workouts)
✅ Successfully trained GMM for DualFlow_2_5.0min
   📊 Clean records: 486
   �� Dropped records: 95

🔄 Training: DualFlow - Difficulty -1 - 5.0min (489 workouts)
✅ Successfully trained GMM for DualFlow_-1_5.0min
   📊 Clean records: 411
   �� Dropped records: 78

🔄 Training: DualFlow - Difficulty -1 - 15.0min (472 workouts)
✅ Successfully trained GMM for DualFlow_

In [10]:
# CELL 7.5: Debug missing values issue
print("🔍 DEBUGGING MISSING VALUES ISSUE")

# Let's check the first combination that failed
game_mode = 'DualFlow'
difficulty = -1
duration_minutes = 10.0

# Filter for this combination
target_data = df_complete[
    (df_complete['game_mode'] == game_mode) &
    (df_complete['difficulty'] == difficulty) &
    (df_complete['duration_minutes'] == duration_minutes)
].copy()

print(f"📊 Original data shape: {target_data.shape}")

# Check missing values in features
features = ['score', 'age', 'startSpeed', 'duration_minutes']
print(f"\n🔍 Missing values in features:")
for feature in features:
    missing_count = target_data[feature].isnull().sum()
    print(f"  {feature}: {missing_count} missing values")

# Check data after dropna()
X = target_data[features].dropna()
print(f"\n📊 Data shape after dropna(): {X.shape}")
print(f"📊 Original shape: {target_data.shape}")
print(f"📊 Difference: {len(target_data) - len(X)} rows dropped")

# Show which columns have missing values
print(f"\n🔍 Detailed missing value analysis:")
for col in target_data.columns:
    missing = target_data[col].isnull().sum()
    if missing > 0:
        print(f"  {col}: {missing} missing values")

🔍 DEBUGGING MISSING VALUES ISSUE
📊 Original data shape: (945, 18)

🔍 Missing values in features:
  score: 0 missing values
  age: 90 missing values
  startSpeed: 0 missing values
  duration_minutes: 0 missing values

📊 Data shape after dropna(): (855, 4)
📊 Original shape: (945, 18)
📊 Difference: 90 rows dropped

🔍 Detailed missing value analysis:
  dob: 90 missing values
  weight: 99 missing values
  height: 102 missing values
  age: 90 missing values
  age_group: 90 missing values


In [11]:
# CELL 8: Save all trained models
import os
import joblib
import json
from datetime import datetime

os.makedirs('models', exist_ok=True)

if 'trained_models' in locals() and trained_models:
    print("💾 Saving all trained models...")

    saved_models = []

    for model_key, model_data in trained_models.items():
        try:
            filename = f"models/{model_key}_gmm_v1.pkl"
            metadata_filename = f"models/{model_key}_gmm_v1_metadata.json"

            joblib.dump(model_data, filename)

            metadata = {
                'model_info': model_data['dataset_info'],
                'training_date': model_data['training_date'],
                'features': model_data['features'],
                'n_clusters': len(model_data['percentile_targets']),
                'cluster_stats': model_data['cluster_stats'],
                'percentile_targets': model_data['percentile_targets']
            }
            with open(metadata_filename, 'w') as f:
                json.dump(metadata, f, indent=2)

            saved_models.append({
                'model_key': model_key,
                'model_file': filename,
                'metadata_file': metadata_filename,
                'workout_count': model_data['dataset_info']['total_records']
            })

            print(f"✅ Saved: {filename}")
            print(f"   Metadata: {metadata_filename}")

        except Exception as e:
            print(f"❌ Error saving {model_key}: {e}")

    print(f"\n🎯 SAVING SUMMARY:")
    print(f"✅ Successfully saved {len(saved_models)} models")

    print(f"\n📁 Saved models:")
    for model in saved_models:
        print(f"  - {model['model_key']} ({model['workout_count']} clean records)")
        print(f"    Model: {model['model_file']}")
        print(f"    Metadata: {model['metadata_file']}")

    summary_data = {
        'training_date': datetime.now().isoformat(),
        'total_models': len(saved_models),
        'models': saved_models
    }
    with open('models/all_models_summary.json', 'w') as f:
        json.dump(summary_data, f, indent=2)

    print(f"\n📋 Summary saved to: models/all_models_summary.json")

else:
    print("❌ No trained models to save. Please run the training cell first.")

💾 Saving all trained models...
✅ Saved: models/DualFlow_-1_10.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_10.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_10.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_10.0min_gmm_v1_metadata.json
✅ Saved: models/LeagueQualification_2_10.0min_gmm_v1.pkl
   Metadata: models/LeagueQualification_2_10.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_5.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_5.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_-1_5.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_5.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_-1_15.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_15.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_3.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_3.0min_gmm_v1_metadata.json
✅ Saved: models/UpperBody_2_5.0min_gmm_v1.pkl
   Metadata: models/UpperBody_2_5.0min_gmm_v1_metadata.json
✅ Saved: models/LeagueQualification_-1_5.0min_gmm_v1.pkl
   Metadata: models/LeagueQualification_-1

In [ ]:
# CELL 8: Save all trained models
import os
import joblib
import json
from datetime import datetime

os.makedirs('models', exist_ok=True)

if 'trained_models' in locals() and trained_models:
    print("💾 Saving all trained models...")

    saved_models = []

    for model_key, model_data in trained_models.items():
        try:
            filename = f"models/{model_key}_gmm_v1.pkl"
            metadata_filename = f"models/{model_key}_gmm_v1_metadata.json"

            joblib.dump(model_data, filename)

            metadata = {
                'model_info': model_data['dataset_info'],
                'training_date': model_data['training_date'],
                'features': model_data['features'],
                'n_clusters': len(model_data['percentile_targets']),
                'cluster_stats': model_data['cluster_stats'],
                'percentile_targets': model_data['percentile_targets']
            }
            with open(metadata_filename, 'w') as f:
                json.dump(metadata, f, indent=2)

            saved_models.append({
                'model_key': model_key,
                'model_file': filename,
                'metadata_file': metadata_filename,
                'workout_count': model_data['dataset_info']['total_records']
            })

            print(f"✅ Saved: {filename}")
            print(f"   Metadata: {metadata_filename}")

        except Exception as e:
            print(f"❌ Error saving {model_key}: {e}")

    print(f"\n🎯 SAVING SUMMARY:")
    print(f"✅ Successfully saved {len(saved_models)} models")

    print(f"\n📁 Saved models:")
    for model in saved_models:
        print(f"  - {model['model_key']} ({model['workout_count']} clean records)")
        print(f"    Model: {model['model_file']}")
        print(f"    Metadata: {model['metadata_file']}")

    summary_data = {
        'training_date': datetime.now().isoformat(),
        'total_models': len(saved_models),
        'models': saved_models
    }
    with open('models/all_models_summary.json', 'w') as f:
        json.dump(summary_data, f, indent=2)

    print(f"\n📋 Summary saved to: models/all_models_summary.json")

else:
    print("❌ No trained models to save. Please run the training cell first.")

💾 Saving all trained models...
✅ Saved: models/DualFlow_-1_10.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_10.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_10.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_10.0min_gmm_v1_metadata.json
✅ Saved: models/LeagueQualification_2_10.0min_gmm_v1.pkl
   Metadata: models/LeagueQualification_2_10.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_5.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_5.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_-1_5.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_5.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_-1_15.0min_gmm_v1.pkl
   Metadata: models/DualFlow_-1_15.0min_gmm_v1_metadata.json
✅ Saved: models/DualFlow_2_3.0min_gmm_v1.pkl
   Metadata: models/DualFlow_2_3.0min_gmm_v1_metadata.json
✅ Saved: models/UpperBody_2_5.0min_gmm_v1.pkl
   Metadata: models/UpperBody_2_5.0min_gmm_v1_metadata.json
✅ Saved: models/LeagueQualification_-1_5.0min_gmm_v1.pkl
   Metadata: models/LeagueQualification_-1